# Object Oriented Code for Tensorflow  Machine Learning on  Diabetes Dataset

In [79]:
try:

    import tensorflow as tf

    # for Data Processing
    import numpy as np
    import pandas as pd

    # for Plotting
    import matplotlib.pyplot as plt
    %matplotlib inline

    # for Data Processing
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import confusion_matrix,classification_report
    import scikitplot as skplt
    
    print('Library Loaded .........')
except:
    print('One or More Library was not Found ! ')


Library Loaded .........


In [71]:
class NeuralNetwork(object):
    def __init__(self,epoch=100,batch_size =30,learning_rate = 0.001):
        self.epoch = epoch
        self.batch_size = batch_size
        self.learning_rate = learning_rate

    def process_data(self):

        """

        :return: X_Train, X_Test, Y_Train, Y_Test
        """

        # Name for the column  or Features Map
        columns_to_named = ["Pregnancies","Glucose","BloodPressure",
                            "SkinThickness","Insulin","BMI","DiabetesPedigreeFunction",
                            "Age","Class"]

        # Read the Dataset and Rename the Column
        df = pd.read_csv("/Users/soumilshah/IdeaProjects/mytensorflow/Dataset/pima-indians-diabetes.csv",header=0,names=columns_to_named)

        col_norm =['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
                   'BMI', 'DiabetesPedigreeFunction']

        # Normalization using Custom Lambda Function

        df1_norm = df[col_norm].apply(lambda x :( (x - x.min()) / (x.max()-x.min()) ) )

        X_Data = df1_norm
        Y_Data = df["Class"]

        self.X_Train, self.X_Test, self.Y_Train, self.Y_Test = train_test_split(X_Data,Y_Data, test_size=0.3,random_state=101)

        return self.X_Train, self.X_Test, self.Y_Train, self.Y_Test

    def create_feature_columns(self):
        """

        :return: feature_column ITS a LIST
        """

        feat_Pregnancies = tf.feature_column.numeric_column('Pregnancies')
        feat_Glucose = tf.feature_column.numeric_column('Glucose')
        feat_BloodPressure = tf.feature_column.numeric_column('BloodPressure')
        feat_SkinThickness_tricep = tf.feature_column.numeric_column('SkinThickness')
        feat_Insulin = tf.feature_column.numeric_column('Insulin')
        feat_BMI = tf.feature_column.numeric_column('BMI')
        feat_DiabetesPedigreeFunction  = tf.feature_column.numeric_column('DiabetesPedigreeFunction')

        self.feature_column = [feat_Pregnancies, feat_Glucose, feat_BloodPressure,
                               feat_SkinThickness_tricep, feat_Insulin,
                               feat_BMI , feat_DiabetesPedigreeFunction]

        return self.feature_column

    def create_input_and_test_func(self):
        
        """
        
        :return: input_func, eval_input_func
        """

        self.X_Train, self.X_Test, self.Y_Train, self.Y_Test = self.process_data()
        self.input_func = tf.estimator.inputs.pandas_input_fn(x=self.X_Train, y=self.Y_Train,
                                                              batch_size=self.batch_size,
                                                              num_epochs =self.epoch,
                                                              shuffle=True)
        self.eval_input_func = tf.estimator.inputs.pandas_input_fn(x=self.X_Test,
                                                                   y=self.Y_Test,
                                                                   batch_size=self.batch_size,
                                                                   num_epochs=1,
                                                                   shuffle=False)
        return self.input_func, self.eval_input_func

    def create_Model(self):
        """
        
        :return: model
        """

        self.feature_column = self.create_feature_columns()
        self.model = tf.estimator.LinearClassifier(feature_columns=self.feature_column,
                                              n_classes=2)
        return self.model

    def train(self,model):
        """
        
        :param model: 
        :return: history
        """
        
        self.input_func, self.eval_input_func = self.create_input_and_test_func()
        self.history = model.train(input_fn=self.input_func, steps = self.epoch)

        return self.history

    def test(self, model):
        """
        
        :param model: 
        :return: results 
        """

        self.input_func, self.eval_input_func = self.create_input_and_test_func()
        self.results = model.evaluate(self.eval_input_func)

        return self.results

In [83]:

if __name__ == "__main__":
    n = NeuralNetwork(epoch=120,batch_size=80,learning_rate=0.1)
    model = n.create_Model()
    history = n.train(model)
    results = n.test(model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/yh/7gktt0ls0fj77fnrs694ht6m0000gn/T/tmpp3mo3yn4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a320b9cc0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create Che

In [84]:
results

{'accuracy': 0.7056277,
 'accuracy_baseline': 0.64935064,
 'auc': 0.7383127,
 'auc_precision_recall': 0.5997658,
 'average_loss': 0.589937,
 'label/mean': 0.35064936,
 'loss': 45.425144,
 'precision': 0.84210527,
 'prediction/mean': 0.35701972,
 'recall': 0.19753087,
 'global_step': 120}